# AFIP Data Cleansing 

In [1]:
import re
import cchardet as chardet
import pandas as pd
import dask.dataframe as dd

In [2]:
def get_encoding_type(file_path):
    rawdata = open(file_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [3]:
file_dir = '../../data/estatutos/external_sources/SELE-SAL-CONSTA.p20out1.20200509.tmp'
get_encoding_type(file_dir)

'ISO-8859-4'

## We read the file with dask in order to create small partitions

We make every transformation required in order to preserve the data, and later perform every transformation.

In [4]:
ddf = dd.read_csv(file_dir, encoding='ISO-8859-4', blocksize=6400, dtype=str)

In [5]:
df = ddf.partitions[-1].compute()

In [6]:
cuit = re.compile(r'(\d{2}(\d{8})\d)')
basura = re.compile(r'[A-Z]+\d+$')

In [7]:
df.columns = ['concatenado']

In [8]:
cuits = df.concatenado.str.extract(cuit)

In [9]:
cuits.columns = ['cuit', 'dni']

In [10]:
cuits

,cuit,dni
0,34600983158,60098315
1,34602466150,60246615
2,34603819499,60381949
3,34604104035,60410403
4,34604160741,60416074
...,...,...
82,34688233358,68823335
83,34999032089,99903208
84,34999230573,99923057
85,34999257560,99925756


In [11]:
df['concatenado'] = df.concatenado.str.replace(cuit, '', regex=True)
df['concatenado'] = df.concatenado.str.replace(basura, '', regex=True)
df

,concatenado
0,CONSORCIO DE PROPIETARIOS
1,FIORDANI DOUGLAS Y FIORDANI
2,CONSORCIO DE PROPIETARIOS AV
3,CONSORCIO DE PROPIETARIOSAVDA
4,JUNCALITO SOCIEDAD COLECTIVA
...,...
82,FANCON SA
83,GOBIERNO DE LA CIUDAD DE
84,DIRECCION GENERAL DE
85,MUNICIPALIDAD DE LA MATANZA


In [12]:
df = pd.concat([df, cuits], axis=1)

#### Final output

In [13]:
df

,concatenado,cuit,dni
0,CONSORCIO DE PROPIETARIOS,34600983158,60098315
1,FIORDANI DOUGLAS Y FIORDANI,34602466150,60246615
2,CONSORCIO DE PROPIETARIOS AV,34603819499,60381949
3,CONSORCIO DE PROPIETARIOSAVDA,34604104035,60410403
4,JUNCALITO SOCIEDAD COLECTIVA,34604160741,60416074
...,...,...,...
82,FANCON SA,34688233358,68823335
83,GOBIERNO DE LA CIUDAD DE,34999032089,99903208
84,DIRECCION GENERAL DE,34999230573,99923057
85,MUNICIPALIDAD DE LA MATANZA,34999257560,99925756


## We restart the process but with the entire dataset

We open the file with pure python since we were having issues with its contents.

In [14]:
with open(file_dir, 'r', encoding='ISO-8859-4') as f:
    text = f.read()

In [15]:
entries = text.split('\n')

In [16]:
df = pd.DataFrame(entries, columns=['concatenado'])

In [17]:
df = dd.from_pandas(df, chunksize=10000)

In [18]:
sucesion = re.compile(r'SUCESION\sDE')

In [19]:
cuits = df.concatenado.str.extract(cuit)
cuits.columns = ['cuit', 'dni']
df['concatenado'] = df.concatenado.str.replace(cuit, '', regex=True)
df['concatenado'] = df.concatenado.str.replace(basura, '', regex=True)
df['concatenado'] = df.concatenado.str.replace(sucesion, '', regex=True)
df = dd.concat([df, cuits], axis=1)

### We save the data to disk

In [20]:
%%time
df.to_csv('../../data/estatutos/external_sources/afip_names_cuits.csv', encoding='UTF-8', single_file=True, index=False)

CPU times: user 54.5 s, sys: 1.11 s, total: 55.6 s
Wall time: 54.9 s


['/home/cpega/projects/ai/jupyter-tests/NLPTools/nlptools/data/estatutos/external_sources/afip_names_cuits.csv']

##### We restart the kernel and read the new file with pandas

In [21]:
from time import sleep
exit()
sleep(30)

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('../../data/estatutos/external_sources/afip_names_cuits.csv', dtype=str)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4955904 entries, 0 to 4955903
Data columns (total 3 columns):
concatenado    object
cuit           object
dni            object
dtypes: object(3)
memory usage: 113.4+ MB


In [25]:
df.head(50)

,concatenado,cuit,dni
0,PERROTTA CAYETANO,20000033481,00003348
1,MULLER CARLOS JORGE,20000035891,00003589
2,CANTARELLA ADELINA,20000179672,00017967
3,GEROSA RICARDO,20000278417,00027841
4,BASAŅES ZAVALLA,20000684830,00068483
5,NUŅEZ JOSE,20000932699,00093269
6,FINCATI MARIA,20000962997,00096299
7,CID LIBRADO,20001067002,00106700
8,MERCOL OSVALDO,20001220986,00122098
9,VALSANGIACOMO,20001430999,00143099
